# project

Note - This code is based on the best HW_3 and HW_4.
These codes consist of 3 main parts.
Part 1: Download the first 10 hours real-time data.
Part 2: Train regression models based on the first 10 hours data.
Part 3: Download new 10-hour real-time data with predicted values, etc.

In [1]:
pip install polygon-api-client==0.2.11

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Install polygon-api-client library

In [3]:
# Import required libraries
import datetime
import time
from polygon import RESTClient
from sqlalchemy import create_engine 
from sqlalchemy import text
import pandas as pd
from math import sqrt
from math import isnan
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from math import floor


In [4]:
# Part 1
# Download 10 hours real-time data

In [5]:
'''
THIS IS MAX CODE
'''
# We can buy, sell, or do nothing each time we make a decision.
# This class defies a nobject for keeping track of our current investments/profits for each currency pair
class portfolio(object):
    def __init__(self,from_,to):
        # Initialize the 'From' currency amont to 1
        self.amount = 1
        self.curr2 = 0
        self.from_ = from_
        self.to = to
        # We want to keep track of state, to see what our next trade should be
        self.Prev_Action_was_Buy = False
    
    # This defines a function to buy the 'To' currency. It will always buy the max amount, in whole number
    # increments
    def buy_curr(self, price):
        if self.amount >= 1:
            num_to_buy = floor(self.amount)
            self.amount -= num_to_buy
            self.Prev_Action_was_Buy = True
            self.curr2 += num_to_buy*price
            print("Bought %d worth of the target currency (%s). Our current profits and losses in the original currency (%s) are: %f." % (num_to_buy,self.to,self.from_,(self.amount-1)))
        else:
            print("There was not enough of the original currency (%s) to make another buy." % self.from_)
    # This defines a function to sell the 'To' currency. It will always sell the max amount, in a whole number
    # increments
    def sell_curr(self, price):
        if self.curr2 >= 1:
            num_to_sell = floor(self.curr2)
            self.amount += num_to_sell * (1/price)
            self.Prev_Action_was_Buy = False
            self.curr2 -= num_to_sell
            print("Sold %d worth of the target currency (%s). Our current profits and losses in the original currency (%s) are: %f." % (num_to_sell,self.to,self.from_,(self.amount-1)))
        else:
            print("There was not enough of the target currency (%s) to make another sell." % self.to)   

In [6]:
'''
Function to calculate 100 Keltner Upper Bands
'''
def calculateKeltnerChannelUB(mean, VOL):
  UB100values = []
  for n in range(1,101):
    UB100values.append((mean+ n*0.025*VOL))
  return UB100values

#function to calculate 100 Keltner Lower Bands
def calculateKeltnerChannelLB(mean, VOL):
  LB100values = []
  for n in range(1,101):
    LB100values.append((mean - n*0.025*VOL))
  return LB100values

'''
Function that counts N i.e the number of times a price crosses a Keltner Channel from lastPrice to currentPrice

Arguments we pass are -
# lastPrice: previous currency price
# currentPrice: current currency price 
# UB (Upper Band): 100 Keltner Upper Bands
# LB (Lower Band): 100 Keltner Lower Bands
'''
def countBandsCrossing(lastPrice, currentPrice, UB, LB):
  '''
  Step 1: We find the startPrice and endPrice so as to cound all the bands lying in between these two prices
          start price is the price which is min of lastPrice and currentPrice and end price is the larger price
  Step 2: We calculate totalBands i.e 200 bands
  Step 3: We calculate filteredTotalBands which are the bands that the price changes crosses
  Step 4: We return the length of filteredTotalBands which is the count of bands crossed from the lastPrice to currentPrice
  
  '''
  # Finding startPrice and endPrice to find all the bands lying in between the given price change
  startPrice = min(lastPrice, currentPrice)
  endPrice = max(lastPrice, currentPrice)

  # totalBands are the 200 (LB + UB) in sorted order
  totalBands = LB[::-1] + UB

  # filteredTotalBands are the bands lying between the startPrice and endPrice
  filteredTotalBands = filter(lambda x: endPrice>x> startPrice, totalBands)

  # In order to return the count of bands crossed we simply find the len(filteredTotalBands)
  return len(list(filteredTotalBands))

#function to calculate FD (Fractal Dimension) using formula FD = N/VOL
#function will return 0 if the VOL is 0 (In order to avoid divide by 0 error) 
def calculateFD(N, VOL):
  if VOL == 0:
    return 0
  else:
    return N/VOL


In [7]:
# Calculates the return according to the formula 𝑟𝑖 = (𝑃𝑖 − 𝑃𝑖−1)⁄(𝑃𝑖−1)

# Arguments:
#   - currentPrice: Current mean price of currency pair
#   - lastPrice: Last mean price of currency pair

# Returns:
#   - return 𝑟𝑖

def calculateReturn(currentPrice, lastPrice):
  return (currentPrice-lastPrice)/lastPrice



# Returns sum of last 10 intervals returnVal

# Arguments:
#   - rows: sql rows object which has last 10 rows from corresponding aggregated table

# Returns:
#   - sum of 𝑟𝑖

def getReturnOfLast10(rows):
  returnVal = 0
  for row in rows:
    if row.returnVal:
      returnVal += row.returnVal
  return returnVal

In [8]:
from ast import Pass
# Function slightly modified from polygon sample code to format the date string 
def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M:%S')

# Function which clears the raw data tables once we have aggregated the data in a 6 minute interval
def reset_raw_data_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("DROP TABLE "+curr[0]+curr[1]+"_raw;"))
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_raw(ticktime text, fxrate  numeric, inserttime text);"))

# This creates a table for storing the raw, unaggregated price data for each currency pair in the SQLite database
def initialize_raw_data_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_raw(ticktime text, fxrate  numeric, inserttime text);"))

            
# This creates a table for storing the (6 min interval) aggregated price data for each currency pair in the SQLite database            
def initialize_aggregated_tables(engine,currency_pairs):
  with engine.begin() as conn:
    for curr in currency_pairs:
      # Initializes aggregate table for all currency pairs with fields inserttime, period, max, min, mean, vol and fd
      conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_agg (inserttime text, period numeric, max numeric, min numeric, mean numeric, vol numeric, fd numeric, returnVal numeric);"))


def aggregate_raw_data_tables(engine, currency_pairs, period, N, currencyCheck):
  # Initialize UB and LB dictionary which has currency pairs as key and stores bands based on mean and vol
  UB = {}
  LB = {}
  with engine.begin() as conn:
    for curr in currency_pairs:
      if currencyCheck[curr[0]+curr[1]] == True:
        with engine.begin() as conn:
          # Calculates mean, max, and min from raw tables which has price data of last 6 mins period
          result = conn.execute(text("SELECT AVG(fxrate) as mean, MIN(fxrate) as min, MAX(fxrate) as max FROM "+curr[0]+curr[1]+"_raw;"))
          for row in result:
            mean = row.mean
            min = row.min
            max = row.max
            vol = (max-min)/mean

          # Calculate UB and LB for the currency pair and storing it into dict
          UB[curr[0]+curr[1]] = calculateKeltnerChannelUB(mean, vol)
          LB[curr[0]+curr[1]] = calculateKeltnerChannelLB(mean, vol)

          date_res = conn.execute(text("SELECT MAX(ticktime) as last_date FROM " +curr[0]+curr[1]+"_raw;"))
          for row in date_res:
            last_date = row.last_date

          # If it is the 1st period, we skip adding FD value in table 
          if period == 1:
            conn.execute(text("INSERT INTO " +curr[0]+curr[1]+"_agg (inserttime, period, max, min, mean, vol) VALUES (:inserttime, :period, :max, :min, :mean, :vol);"),[{"inserttime":last_date, "period": period, "max": max, "min": min, "mean": mean, "vol": vol}])
          else:
            fd = calculateFD(N[curr[0]+curr[1]],vol)
            result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_agg ORDER BY rowid DESC LIMIT 1;"))
            for row in result:
              lastMean = row.mean
            returnVal = calculateReturn(mean, lastMean)
            conn.execute(text("INSERT INTO " +curr[0]+curr[1]+"_agg (inserttime, period, max, min, mean, vol, fd, returnVal) VALUES (:inserttime, :period, :max, :min, :mean, :vol, :fd, :returnVal );"),[{"inserttime":last_date, "period": period, "max": max, "min": min, "mean": mean, "vol": vol, "fd": fd, "returnVal":returnVal}])

  # Returning Keltner Channel Upper bound and Lower bound to the main function
  return UB, LB

In [9]:

# This creates an output table for storing the data after 60 min interval for each currency pair in the SQLite database     
# The table has window, balance, returnVal and position fields 
def initialize_output_tables(engine,currency_pairs):
  with engine.begin() as conn:
    for curr in currency_pairs:
      # Initializes aggregate table for all currency pairs with fields inserttime, period, max, min, mean, vol and fd
      conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_output (window numeric, balance numeric, returnVal numeric, position text);"))


In [10]:
def fill_output_data_tables(engine, currency_pairs, window, currencyCheck):
  '''
    Function called every 60 minutes to make a decision on current open positions and fill the output data tables
    
    Arguments:
      - engine: Engine object from sqlalchemy
      - currency_pairs: Nested list of each currency pair
      - window: current window count
      - currencyCheck: It is a dictionary which has keys as currencyPair and values as True or False
    Returns:
      - currencyCheck: Updated currencyCheck dict
  '''

  #longCurrency List (BUY)
  longCurrency = ["EURUSD", "GBPEUR", "USDCHF", "USDCAD"]
  #shortCurrency List (SELL)
  shortCurrency = [ "USDHKD", "USDAUD", "USDNZD", "USDSGD"]


  for curr in currency_pairs:
    # We will check if currency pair is set to True in the currencyCheck dict which will let us know weather to close the position or continue
    if currencyCheck[curr[0]+curr[1]] == True:

      # setting position based on longCurrency list
      if curr[0]+curr[1] in longCurrency:
        position = 'LONG'
      else:
        position = 'SHORT'

      with engine.begin() as conn:
        # Fetching last 10 rows from aggregate table to get the return value
        result = conn.execute(text("SELECT *  FROM "+curr[0]+curr[1]+"_agg ORDER BY rowid DESC LIMIT 10;"))
        # R10 stores the sum of last 10 returnVal 
        R10 = getReturnOfLast10(result)

        # At T10, cutoff value to use is 0.250%
        if window == 1:
          balance = 100
          if position == 'LONG':
            # Long condition -> , a profitable trade has a positive return but we have a tolarence of 0.250%
            # 0.250% = 0.0025
            if R10 >= -0.0025:
              balance = balance + 100 + R10
            # If not profitable, we will close the position and set the currencyCheck flag to false
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False
          else:
            # Short condition -> a profitable trade has a negative return. 
            if R10 <= 0.0025:
              balance = balance + 100 + R10
            # If not profitable, we will close the position and set the currencyCheck flag to false
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False
        
        # At T20, cutoff value to use is 0.150%
        elif window == 2:
          # The below line fetches the last balance of the currency pair of last 60 min period
          result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_output WHERE window = " + str(window-1)+";"))
          for row in result:
            balance = row.balance

          if position == 'LONG':
            if R10 >= -0.0015:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False
          else:
            # Short condition
            if R10 <= 0.0015:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False


        # At T30, value to use is 0.100%
        elif window == 3:
          result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_output WHERE window = " + str(window-1)+";"))
          for row in result:
            balance = row.balance

          if position == 'LONG':
            if R10 >= -0.001:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False
          else:
            if R10 <= 0.001:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False


        # At T40, value to use is 0.050%
        elif window == 4:
          result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_output WHERE window = " + str(window-1)+";"))
          for row in result:
            balance = row.balance

          if position == 'LONG':
            if R10 >= -0.0005:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False
          else:
            if R10 <= 0.0005:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False


        # After T40, value to use is 0.050%
        elif window > 4:
          result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_output WHERE window = " + str(window-1)+";"))
          for row in result:
            balance = row.balance

          if position == 'LONG':
            if R10 >= -0.0005:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False
          else:
            if R10 <= 0.0005:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False
          
        conn.execute(text("INSERT INTO " +curr[0]+curr[1]+"_output (window, balance, returnVal, position) VALUES (:window, :balance, :returnVal, :position);"),[{"window":window, "balance": balance, "returnVal": R10, "position": position}])

  return currencyCheck


In [11]:
# This main function repeatedly calls the polygon api every 1 seconds for 24 hours 
# and stores the results.
def main(currency_pairs):
    # The api key given by the professor is fetched from the library imported in the 1st cell 
    key = "beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq"
    
    # Number of list iterations - each one should last about 1 second
    count = 0
    agg_count = 0
    # Counter to keep track of 10 intervals which is after every 60 minutes
    decision_count = 0

    # currencyCheck dictionary has keys as currencyPair and values as True or False
    # By default all are set to True but if we decide not to invest more and stop the position, we set this to False
    currencyCheck = {}
    
    # Create an engine to connect to the database; setting echo to false should stop it from logging in std.out
    engine = create_engine("sqlite:///final.db", echo=False, future=True)
    
    # Create the needed tables in the database
    initialize_raw_data_tables(engine,currency_pairs)
    initialize_aggregated_tables(engine,currency_pairs)
    # Create the output tables for all currency pairs
    initialize_output_tables(engine,currency_pairs)
    
    # Open a RESTClient for making the api calls
    client = RESTClient(key)
   
    # counter for period 
    period_count = 0 
    # counter for window 
    window_count = 0
    # dictionary to store N values for all currency pairs
    N = {}
    UB = {}
    LB = {}

    # stores last price of currencypairs
    lastPrice = {}

    # Loop that runs until the total duration of the program hits 10 hours. 
    # 36000 seconds = 10 hours
    while count < 36000:
      
      # Make a check to see if 6 minutes has been reached or not
      if agg_count == 360:
        # Aggregate the data and clear the raw data tables
        period_count+=1
        UB, LB = aggregate_raw_data_tables(engine, currency_pairs, period_count, N, currencyCheck)
        reset_raw_data_tables(engine,currency_pairs)
        agg_count = 0
        N = {}
        lastPrice = {}
        
      # Only call the api every 1 second, so wait here for 0.75 seconds, because the 
      # code takes about .15 seconds to run
      time.sleep(0.55)

      # Only called when 10 periods of 6 mins are done i.e after every 60 mins
      if decision_count == 3600:
        window_count +=1
        currencyCheck = fill_output_data_tables(engine, currency_pairs, window_count, currencyCheck)
        decision_count = 0
      
      # Increment the counters
      count += 1
      agg_count +=1
      decision_count +=1

      # Loop through each currency pair
      for currency in currency_pairs:
        # Set the input variables to the API
        from_ = currency[0]
        to = currency[1]

        # initializing currencyCheck for all currency pair with True
        if from_+to not in currencyCheck:
          currencyCheck[from_+to] = True

        # Run the below logic only when currencyCheck of the corresponding currency pair is set to True 
        if currencyCheck[from_+to] == True:
          # Call the API with the required parameters
          try:
              resp = client.forex_currencies_real_time_currency_conversion(from_, to, amount=100, precision=2)
          except:
              continue

          # This gets the Last Trade object defined in the API Resource
          last_trade = resp.last

          # Format the timestamp from the result
          dt = ts_to_datetime(last_trade["timestamp"])

          # Get the current time and format it
          insert_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
          
          # Calculate the price by taking the average of the bid and ask prices
          avg_price = (last_trade['bid'] + last_trade['ask'])/2

          # In the 1st period, UB and LB will be an empty dictionary so this condition will be false for 1st period
          if UB!={} and LB!={}:

            # From 2nd period to 100th period
            # if currency[0]+currency[1] key is in lastPrice, it means we have a last price to send in the countBandsCrossing() function and we update lastPrice with the current price which is the avg_price
            if currency[0]+currency[1] in lastPrice:
              N[currency[0]+currency[1]] += countBandsCrossing(lastPrice[currency[0]+currency[1]], avg_price, UB[currency[0]+currency[1]], LB[currency[0]+currency[1]])    
              lastPrice[currency[0]+currency[1]] = avg_price
            
            # if currency[0]+currency[1] key not in lastPrice and N, we initialize here
            else:
              lastPrice[currency[0]+currency[1]] = avg_price
              N[currency[0]+currency[1]] = 0

          # Write the data to the SQLite database, raw data tables
          with engine.begin() as conn:
              conn.execute(text("INSERT INTO "+from_+to+"_raw(ticktime, fxrate, inserttime) VALUES (:ticktime, :fxrate, :inserttime)"),[{"ticktime": dt, "fxrate": avg_price, "inserttime": insert_time}])

In [12]:
# A dictionary defining the set of currency pairs we will be pulling data for
currency_pairs = [["EUR","USD",[],portfolio("EUR","USD")],
                  ["GBP","USD",[],portfolio("GBP","USD")],
                  ["USD","CHF",[],portfolio("USD","CHF")],
                  ["USD","CAD",[],portfolio("USD","CAD")],
                  ["USD","HKD",[],portfolio("USD","HKD")],
                  ["USD","AUD",[],portfolio("USD","AUD")],
                  ["USD","NZD",[],portfolio("USD","NZD")],
                  ["USD","SGD",[],portfolio("USD","SGD")]]

# Run the main data collection loop
main(currency_pairs)

OperationalError: (sqlite3.OperationalError) table EURUSD_raw already exists
[SQL: CREATE TABLE EURUSD_raw(ticktime text, fxrate  numeric, inserttime text);]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [13]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('final.db', isolation_level=None,
                       detect_types=sqlite3.PARSE_COLNAMES)
# make the first return = 0
# Code to convert db tables into csv files


for curr in currency_pairs:
  sql = "SELECT inserttime, period, max, min, mean, vol, fd, IFNULL(returnVal,0)as returnVal FROM "+curr[0]+curr[1]+"_agg"
  db_df = pd.read_sql_query(sql, conn)
  path = curr[0]+curr[1]+'_agg.csv'
  db_df.to_csv(path, index=False)

for curr in currency_pairs:
  sql = "SELECT * FROM "+curr[0]+curr[1]+"_output"
  db_df = pd.read_sql_query(sql, conn)
  path = curr[0]+curr[1]+'_output.csv'
  db_df.to_csv(path, index=False)

In [14]:
# Part 2
# Train regression model

In [15]:
# install pycaret
!pip install pycaret

In [16]:
# read data from csv 
# download of 10 hours of data
columns = ["inserttime","period", "max", "min", "mean", "vol", "fd","returnVal"]
EURUSD_df = pd.read_csv('/Users/82545/Desktop/data_csv/EURUSD_agg.csv', names=columns, na_values='?', header=0,dtype={"inserttime":'str',"period":'int', "max":'float64', "min":'float64',"mean":'float64', "vol":'float64', "fd":'float64',"returnVal":'float64'})
GBPUSD_df = pd.read_csv('/Users/82545/Desktop/data_csv/GBPUSD_agg.csv', names=columns, na_values='?', header=0)
USDCHF_df = pd.read_csv('/Users/82545/Desktop/data_csv/USDCHF_agg.csv', names=columns, na_values='?', header=0)
USDCAD_df = pd.read_csv('/Users/82545/Desktop/data_csv/USDCAD_agg.csv', names=columns, na_values='?', header=0)
USDHKD_df = pd.read_csv('/Users/82545/Desktop/data_csv/USDHKD_agg.csv', names=columns, na_values='?', header=0)
USDAUD_df = pd.read_csv('/Users/82545/Desktop/data_csv/USDAUD_agg.csv', names=columns, na_values='?', header=0)
USDNZD_df = pd.read_csv('/Users/82545/Desktop/data_csv/USDNZD_agg.csv', names=columns, na_values='?', header=0)
USDSGD_df = pd.read_csv('/Users/82545/Desktop/data_csv/USDSGD_agg.csv', names=columns, na_values='?', header=0)


In [17]:
# match the currency pairs and data frame
currency_pairs_dict = {"EURUSD": EURUSD_df, "GBPUSD": GBPUSD_df, "USDCHF": USDCHF_df, "USDCAD": USDCAD_df,
                       "USDHKD": USDHKD_df, "USDAUD": USDAUD_df, "USDNZD": USDNZD_df, "USDSGD": USDSGD_df}

In [18]:
# classify the data by vol/FD
# set them as 1=low, 2=medium, 3=high
# first bar = top = 33
# second bar = top = 67
def classify(currency, original_df, column_name,first_bar=33, second_bar=67):
    df = original_df.copy(deep=True)
    # sort by vol
    df = df.sort_values(by=[column_name])
    # first_bar_dict[column_name]
    for i, row in df.iterrows():
        if i < first_bar:
            df.at[i,column_name] = 1
        elif i < second_bar:
            df.at[i,column_name] = 2
        else:
            df.at[i,column_name] = 3
    return df

In [19]:
formatted_currency_pairs_dict={}
# classify all the fds
for key, df in currency_pairs_dict.items():
    # classify by fd
    df=classify(key,df,"fd")
    # classify by vol
    df=classify(key,df,"vol")
    # classify by mean
    df=classify(key,df,"mean")
    # classify by min
    df=classify(key,df,"min")
    # classify by max
    df=classify(key,df,"max")
    # show the results to check
    print(df)
    formatted_currency_pairs_dict[key]=df

             inserttime  period  max  min  mean  vol   fd  returnVal
85  2022-12-19 08:27:56      86  3.0  3.0   3.0  3.0  3.0  -0.000217
86  2022-12-19 08:34:28      87  3.0  3.0   3.0  3.0  3.0   0.000076
84  2022-12-19 08:21:41      85  3.0  3.0   3.0  3.0  3.0  -0.000143
87  2022-12-19 08:40:50      88  3.0  3.0   3.0  3.0  3.0   0.000092
97  2022-12-19 09:50:08      98  3.0  3.0   3.0  3.0  3.0  -0.000071
..                  ...     ...  ...  ...   ...  ...  ...        ...
45  2022-12-19 04:13:30      46  2.0  2.0   2.0  2.0  2.0   0.000440
49  2022-12-19 04:38:49      50  2.0  2.0   2.0  2.0  2.0  -0.000362
51  2022-12-19 04:51:27      52  2.0  2.0   2.0  2.0  2.0   0.000013
48  2022-12-19 04:32:35      49  2.0  2.0   2.0  2.0  2.0  -0.000004
47  2022-12-19 04:25:56      48  2.0  2.0   2.0  2.0  2.0   0.000598

[99 rows x 8 columns]
             inserttime  period  max  min  mean  vol   fd  returnVal
86  2022-12-19 08:34:28      87  3.0  3.0   3.0  3.0  3.0  -0.000023
87  2022-12

In [20]:
from pycaret import regression

In [21]:
# extract features we want to use to train the model
def extract_features(df):
    df=df[["mean","fd","vol","max","min","returnVal"]]
    return df

In [22]:
# train model for all currency df
def train_regression_model(df, target):
    regression.setup(data=df,target=target,fold_shuffle=True)
    best=regression.compare_models()
    regression.evaluate_model(best)
    return best

In [23]:
currency_model_dict={}
for curreny, df in formatted_currency_pairs_dict.items():
    # extract feature
    df=extract_features(df)
    currency_model_dict[curreny]=train_regression_model(df,'returnVal')

,Description,Value
0,Session id,8023
1,Target,returnVal
2,Target type,Regression
3,Data shape,"(99, 6)"
4,Train data shape,"(69, 6)"
5,Test data shape,"(30, 6)"
6,Numeric features,5
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
par,Passive Aggressive Regressor,0.0002,0.0000,0.0003,-0.2299,0.0003,1.0000,0.0080
llar,Lasso Least Angle Regression,0.0002,0.0000,0.0003,-0.2993,0.0003,1.4802,0.0090
lasso,Lasso Regression,0.0002,0.0000,0.0003,-0.2993,0.0003,1.4802,0.1860
dummy,Dummy Regressor,0.0002,0.0000,0.0003,-0.2993,0.0003,1.4802,0.0080
en,Elastic Net,0.0002,0.0000,0.0003,-0.2993,0.0003,1.4802,0.0080
ridge,Ridge Regression,0.0002,0.0000,0.0003,-0.3001,0.0003,4.3343,0.0090
br,Bayesian Ridge,0.0002,0.0000,0.0003,-0.3003,0.0003,4.3534,0.0090
lr,Linear Regression,0.0002,0.0000,0.0003,-0.3004,0.0003,4.3549,0.4890
lar,Least Angle Regression,0.0002,0.0000,0.0003,-0.3004,0.0003,4.3549,0.0090
huber,Huber Regressor,0.0002,0.0000,0.0003,-0.3173,0.0003,3.0291,0.0100


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

,Description,Value
0,Session id,4567
1,Target,returnVal
2,Target type,Regression
3,Data shape,"(99, 6)"
4,Train data shape,"(69, 6)"
5,Test data shape,"(30, 6)"
6,Numeric features,5
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


...

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

,Description,Value
0,Session id,4778
1,Target,returnVal
2,Target type,Regression
3,Data shape,"(99, 6)"
4,Train data shape,"(69, 6)"
5,Test data shape,"(30, 6)"
6,Numeric features,5
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
par,Passive Aggressive Regressor,0.0001,0.0000,0.0002,-0.3217,0.0002,1.0000,0.0090
omp,Orthogonal Matching Pursuit,0.0001,0.0000,0.0002,-0.4178,0.0002,1.0742,0.0100
lasso,Lasso Regression,0.0001,0.0000,0.0002,-0.4178,0.0002,1.0742,0.0080
dummy,Dummy Regressor,0.0001,0.0000,0.0002,-0.4178,0.0002,1.0742,0.0080
llar,Lasso Least Angle Regression,0.0001,0.0000,0.0002,-0.4178,0.0002,1.0742,0.0080
en,Elastic Net,0.0001,0.0000,0.0002,-0.4178,0.0002,1.0742,0.0070
lightgbm,Light Gradient Boosting Machine,0.0001,0.0000,0.0002,-0.4492,0.0002,1.1115,0.0100
ridge,Ridge Regression,0.0001,0.0000,0.0002,-0.5212,0.0001,1.1708,0.0070
lr,Linear Regression,0.0001,0.0000,0.0002,-0.5220,0.0001,1.1718,0.0240
lar,Least Angle Regression,0.0001,0.0000,0.0002,-0.5220,0.0001,1.1718,0.0080


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [24]:
# perdict test

In [25]:
# read data from csv 

columns = ["inserttime","period", "max", "min", "mean", "vol", "fd","returnVal"]
EURUSD_test_df = pd.read_csv('/Users/82545/Desktop/data_csv/36/EURUSD_agg.csv', names=columns, na_values='?', header=0,dtype={"inserttime":'str',"period":'int', "max":'float64', "min":'float64',"mean":'float64', "vol":'float64', "fd":'float64',"returnVal":'float64'})
GBPUSD_test_df = pd.read_csv('/Users/82545/Desktop/data_csv/36/GBPUSD_agg.csv', names=columns, na_values='?', header=0)
USDCHF_test_df = pd.read_csv('/Users/82545/Desktop/data_csv/36/USDCHF_agg.csv', names=columns, na_values='?', header=0)
USDCAD_test_df = pd.read_csv('/Users/82545/Desktop/data_csv/36/USDCAD_agg.csv', names=columns, na_values='?', header=0)
USDHKD_test_df = pd.read_csv('/Users/82545/Desktop/data_csv/36/USDHKD_agg.csv', names=columns, na_values='?', header=0)
USDAUD_test_df = pd.read_csv('/Users/82545/Desktop/data_csv/36/USDAUD_agg.csv', names=columns, na_values='?', header=0)
USDNZD_test_df = pd.read_csv('/Users/82545/Desktop/data_csv/36/USDNZD_agg.csv', names=columns, na_values='?', header=0)
USDSGD_test_df = pd.read_csv('/Users/82545/Desktop/data_csv/36/USDSGD_agg.csv', names=columns, na_values='?', header=0)

In [26]:
# match the currency pairs and data frame
currency_pairs_test_dict = {"EURUSD": EURUSD_test_df, "GBPUSD": GBPUSD_test_df, "USDCHF": USDCHF_test_df, "USDCAD": USDCAD_test_df,
                       "USDHKD": USDHKD_test_df, "USDAUD": USDAUD_test_df, "USDNZD": USDNZD_test_df, "USDSGD": USDSGD_test_df}

In [27]:
def classify_test_data(currency,column_name):
    # get original df
    train_df=currency_pairs_dict[currency]
    # sort by vol
    train_df=train_df.sort_values(by=[column_name])
    # get first bar and second bar
    first=train_df.at[12,column_name]
    second=train_df.at[24,column_name]
    for i, row in currency_pairs_test_dict[currency].iterrows():
        if i < first:
            df.at[i,column_name] = 1
        elif i<second:
            df.at[i,column_name]=2
        else:
            df.at[i,column_name] = 3

In [28]:
# classify test df
for key,df in currency_pairs_test_dict.items():
    # classify by fd
    classify_test_data(key,"fd")
    # classify by vol
    classify_test_data(key,"vol")
    # classify by mean
    classify_test_data(key,"mean")
    # classify by min
    classify_test_data(key,"min")
    # classify by max
    classify_test_data(key,"max")
    
    # show the results to check
    currency_pairs_test_dict[key]=df

In [29]:
print(currency_pairs_test_dict["EURUSD"])

             inserttime  period  max  min  mean  vol   fd  returnVal
0   2022-12-12 19:41:29       1  1.0  1.0   1.0  1.0  1.0   0.000000
1   2022-12-12 19:46:34       2  1.0  1.0   1.0  3.0  1.0   0.000161
2   2022-12-12 19:51:34       3  3.0  3.0   3.0  3.0  1.0  -0.000050
3   2022-12-12 19:56:51       4  3.0  3.0   3.0  3.0  1.0  -0.000167
4   2022-12-12 20:02:04       5  3.0  3.0   3.0  3.0  1.0   0.000157
5   2022-12-12 20:07:12       6  3.0  3.0   3.0  3.0  1.0  -0.000202
6   2022-12-12 20:12:49       7  3.0  3.0   3.0  3.0  1.0   0.000089
7   2022-12-12 20:19:38       8  3.0  3.0   3.0  3.0  1.0   0.000166
8   2022-12-12 20:26:07       9  3.0  3.0   3.0  3.0  1.0  -0.000038
9   2022-12-12 20:32:15      10  3.0  3.0   3.0  3.0  1.0  -0.000129
10  2022-12-12 20:37:45      11  3.0  3.0   3.0  3.0  1.0  -0.000190
11  2022-12-12 20:43:34      12  3.0  3.0   3.0  3.0  1.0   0.000182
12  2022-12-12 20:49:20      13  3.0  3.0   3.0  3.0  1.0   0.000310
13  2022-12-12 20:54:54      14  3

In [30]:
# extract feature to test
def extract_test_features(df):
    return df[["mean","fd","vol","min","max"]]

In [31]:
currency_perdictions_dict={}
# perdict
for key,value in currency_model_dict.items():
    test_df = extract_test_features(currency_pairs_test_dict[key])
    predictions = regression.predict_model(value,data = test_df)
    currency_perdictions_dict[key]= predictions
    

In [33]:
# extract and join actual and perdicted return
output_dict={}
for key, df in currency_perdictions_dict.items():
    # extract label
    perdict_return = df["prediction_label"]
    inserttime=currency_pairs_test_dict[key]["inserttime"]
    period=currency_pairs_test_dict[key]["period"]
    max=currency_pairs_test_dict[key]["max"]
    min=currency_pairs_test_dict[key]["min"]
    mean=currency_pairs_test_dict[key]["mean"]
    vol=currency_pairs_test_dict[key]["vol"]
    fd=currency_pairs_test_dict[key]["fd"]
    acutal_return=currency_pairs_test_dict[key]["returnVal"]
    out=pd.concat([inserttime,period,max,min,mean,vol,fd,perdict_return, acutal_return], axis=1)
    output_dict[key]=out

In [34]:
# write to csv b
for key,df in output_dict.items():
    df["error"]=df['prediction_label']-df['returnVal']
    df.to_csv('/Users/82545/Desktop/data_csv/1/'+key+".csv",header=['inserttime','period', 'max', 'min', 'mean', 'vol', 'fd','perdicted_return','acutal_return','error'])

In [35]:
# Part 3
# predict for next 10 hours 

In [36]:
'''
THIS IS MAX CODE
'''
# We can buy, sell, or do nothing each time we make a decision.
# This class defies a nobject for keeping track of our current investments/profits for each currency pair
class portfolio(object):
    def __init__(self,from_,to):
        # Initialize the 'From' currency amont to 1
        self.amount = 1
        self.curr2 = 0
        self.from_ = from_
        self.to = to
        # We want to keep track of state, to see what our next trade should be
        self.Prev_Action_was_Buy = False
    
    # This defines a function to buy the 'To' currency. It will always buy the max amount, in whole number
    # increments
    def buy_curr(self, price):
        if self.amount >= 1:
            num_to_buy = floor(self.amount)
            self.amount -= num_to_buy
            self.Prev_Action_was_Buy = True
            self.curr2 += num_to_buy*price
            print("Bought %d worth of the target currency (%s). Our current profits and losses in the original currency (%s) are: %f." % (num_to_buy,self.to,self.from_,(self.amount-1)))
        else:
            print("There was not enough of the original currency (%s) to make another buy." % self.from_)
    # This defines a function to sell the 'To' currency. It will always sell the max amount, in a whole number
    # increments
    def sell_curr(self, price):
        if self.curr2 >= 1:
            num_to_sell = floor(self.curr2)
            self.amount += num_to_sell * (1/price)
            self.Prev_Action_was_Buy = False
            self.curr2 -= num_to_sell
            print("Sold %d worth of the target currency (%s). Our current profits and losses in the original currency (%s) are: %f." % (num_to_sell,self.to,self.from_,(self.amount-1)))
        else:
            print("There was not enough of the target currency (%s) to make another sell." % self.to)   

In [37]:
'''
Function to calculate 100 Keltner Upper Bands
'''
def calculateKeltnerChannelUB(mean, VOL):
  UB100values = []
  for n in range(1,101):
    UB100values.append((mean+ n*0.025*VOL))
  return UB100values

#function to calculate 100 Keltner Lower Bands
def calculateKeltnerChannelLB(mean, VOL):
  LB100values = []
  for n in range(1,101):
    LB100values.append((mean - n*0.025*VOL))
  return LB100values

'''
Function that counts N i.e the number of times a price crosses a Keltner Channel from lastPrice to currentPrice

Arguments we pass are -
# lastPrice: previous currency price
# currentPrice: current currency price 
# UB (Upper Band): 100 Keltner Upper Bands
# LB (Lower Band): 100 Keltner Lower Bands
'''
def countBandsCrossing(lastPrice, currentPrice, UB, LB):
  '''
  Step 1: We find the startPrice and endPrice so as to cound all the bands lying in between these two prices
          start price is the price which is min of lastPrice and currentPrice and end price is the larger price
  Step 2: We calculate totalBands i.e 200 bands
  Step 3: We calculate filteredTotalBands which are the bands that the price changes crosses
  Step 4: We return the length of filteredTotalBands which is the count of bands crossed from the lastPrice to currentPrice
  
  '''
  # Finding startPrice and endPrice to find all the bands lying in between the given price change
  startPrice = min(lastPrice, currentPrice)
  endPrice = max(lastPrice, currentPrice)

  # totalBands are the 200 (LB + UB) in sorted order
  totalBands = LB[::-1] + UB

  # filteredTotalBands are the bands lying between the startPrice and endPrice
  filteredTotalBands = filter(lambda x: endPrice>x> startPrice, totalBands)

  # In order to return the count of bands crossed we simply find the len(filteredTotalBands)
  return len(list(filteredTotalBands))

#function to calculate FD (Fractal Dimension) using formula FD = N/VOL
#function will return 0 if the VOL is 0 (In order to avoid divide by 0 error) 
def calculateFD(N, VOL):
  if VOL == 0:
    return 0
  else:
    return N/VOL


In [38]:
# Calculates the return according to the formula 𝑟𝑖 = (𝑃𝑖 − 𝑃𝑖−1)⁄(𝑃𝑖−1)

# Arguments:
#   - currentPrice: Current mean price of currency pair
#   - lastPrice: Last mean price of currency pair

# Returns:
#   - return 𝑟𝑖

def calculateReturn(currentPrice, lastPrice):
  return (currentPrice-lastPrice)/lastPrice



# Returns sum of last 10 intervals returnVal

# Arguments:
#   - rows: sql rows object which has last 10 rows from corresponding aggregated table

# Returns:
#   - sum of 𝑟𝑖

def getReturnOfLast10(rows):
  returnVal = 0
  for row in rows:
    if row.returnVal:
      returnVal += row.returnVal
  return returnVal
# Return_predict
# sum of 10 return_predict

def getReturnOfLast10_pre(rows):
  returnVal_predict = 0
  for row in rows:
    if row.returnVal_predict:
      returnVal_predict += row.returnVal_predict
  return returnVal_predict


In [39]:
from ast import Pass
# Function slightly modified from polygon sample code to format the date string 
def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M:%S')

# Function which clears the raw data tables once we have aggregated the data in a 6 minute interval
def reset_raw_data_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("DROP TABLE "+curr[0]+curr[1]+"_raw;"))
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_raw(ticktime text, fxrate  numeric, inserttime text);"))

# This creates a table for storing the raw, unaggregated price data for each currency pair in the SQLite database
def initialize_raw_data_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_raw(ticktime text, fxrate  numeric, inserttime text);"))

            
# This creates a table for storing the (6 min interval) aggregated price data for each currency pair in the SQLite database            
def initialize_aggregated_tables(engine,currency_pairs):
  with engine.begin() as conn:
    for curr in currency_pairs:
      # Initializes aggregate table for all currency pairs with fields inserttime, period, max, min, mean, vol and fd
      conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_agg (inserttime text, period numeric, max numeric, min numeric, mean numeric, vol numeric, fd numeric, returnVal numeric,returnVal_predict numeric, error numeric);"))


def aggregate_raw_data_tables(engine, currency_pairs, period, N, currencyCheck):
  # Initialize UB and LB dictionary which has currency pairs as key and stores bands based on mean and vol
  UB = {}
  LB = {}
  with engine.begin() as conn:
    for curr in currency_pairs:
      if currencyCheck[curr[0]+curr[1]] == True:
        with engine.begin() as conn:
          # Calculates mean, max, and min from raw tables which has price data of last 6 mins period
          result = conn.execute(text("SELECT AVG(fxrate) as mean, MIN(fxrate) as min, MAX(fxrate) as max FROM "+curr[0]+curr[1]+"_raw;"))
          for row in result:
            mean = row.mean
            min = row.min
            max = row.max
            vol = (max-min)/mean

          # Calculate UB and LB for the currency pair and storing it into dict
          UB[curr[0]+curr[1]] = calculateKeltnerChannelUB(mean, vol)
          LB[curr[0]+curr[1]] = calculateKeltnerChannelLB(mean, vol)

          date_res = conn.execute(text("SELECT MAX(ticktime) as last_date FROM " +curr[0]+curr[1]+"_raw;"))
          for row in date_res:
            last_date = row.last_date

          # If it is the 1st period, we skip adding FD value in table 
          if period == 1:
            conn.execute(text("INSERT INTO " +curr[0]+curr[1]+"_agg (inserttime, period, max, min, mean, vol) VALUES (:inserttime, :period, :max, :min, :mean, :vol);"),[{"inserttime":last_date, "period": period, "max": max, "min": min, "mean": mean, "vol": vol}])
          else:
            fd = calculateFD(N[curr[0]+curr[1]],vol)
            result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_agg ORDER BY rowid DESC LIMIT 1;"))
            for row in result:
              lastMean = row.mean
            returnVal = calculateReturn(mean, lastMean)
            # I cannot define retunrVal_predict in database table, so I made error = 0.00001 here.
            returnVal_predict = returnVal + 0.00001
            error_ = 0.00001
            
# Here is my attempt to write the regression values into the database， but I failed 
# From the best HW4, I learned how to do this towards dataframe, but I cannot do the same thing towards database（Real-time data）.
#           result1 = conn.execute(text("SELECT mean,vol,fd FROM "+curr[0]+curr[1]+"_agg ORDER BY rowid DESC LIMIT 1;"))
#           for row in result1:  
#             returnVal_predict = regression.predict_model(row)
#             error_ = returnVal - returnVal_predict
         
            
            conn.execute(text("INSERT INTO " +curr[0]+curr[1]+"_agg (inserttime, period, max, min, mean, vol, fd, returnVal, returnVal_predict, error) VALUES (:inserttime, :period, :max, :min, :mean, :vol, :fd, :returnVal, :returnVal_predict, :error );"),[{"inserttime":last_date, "period": period, "max": max, "min": min, "mean": mean, "vol": vol, "fd": fd, "returnVal" :returnVal,"returnVal_predict":returnVal_predict, "error":error_}])
    
    
  # Returning Keltner Channel Upper bound and Lower bound to the main function
  return UB, LB



In [40]:
# This creates an output table for storing the data after 60 min interval for each currency pair in the SQLite database     
# The table has window, balance, returnVal and position fields 
def initialize_output_tables(engine,currency_pairs):
  with engine.begin() as conn:
    for curr in currency_pairs:
      # Initializes aggregate table for all currency pairs with fields inserttime, period, max, min, mean, vol and fd
      conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_output (window numeric, balance numeric, returnVal numeric, position text, balance_pre numeric, returnVal_predict numeric);"))

In [41]:
def fill_output_data_tables(engine, currency_pairs, window, currencyCheck):
  '''
    Function called every 60 minutes to make a decision on current open positions and fill the output data tables
    
    Arguments:
      - engine: Engine object from sqlalchemy
      - currency_pairs: Nested list of each currency pair
      - window: current window count
      - currencyCheck: It is a dictionary which has keys as currencyPair and values as True or False
    Returns:
      - currencyCheck: Updated currencyCheck dict
  '''

  #longCurrency List (BUY)
  longCurrency = ["EURUSD", "GBPEUR", "USDCHF", "USDCAD"]
  #shortCurrency List (SELL)
  shortCurrency = [ "USDHKD", "USDAUD", "USDNZD", "USDSGD"]


  for curr in currency_pairs:
    # We will check if currency pair is set to True in the currencyCheck dict which will let us know weather to close the position or continue
    if currencyCheck[curr[0]+curr[1]] == True:

      # setting position based on longCurrency list
      if curr[0]+curr[1] in longCurrency:
        position = 'LONG'
      else:
        position = 'SHORT'
    

      with engine.begin() as conn:
        # Fetching last 10 rows from aggregate table to get the return value
        result = conn.execute(text("SELECT *  FROM "+curr[0]+curr[1]+"_agg ORDER BY rowid DESC LIMIT 10;"))
        # R10 stores the sum of last 10 returnVal 
        R10 = getReturnOfLast10(result)
        # R10_predict stores the sum of last 10 returnVal_predict 
        R10_predict = getReturnOfLast10_pre(result)

        
        # At T10, cutoff value to use is 0.250%
        if window == 1:
          balance = 100
          if position == 'LONG':
            # Long condition -> , a profitable trade has a positive return but we have a tolarence of 0.250%
            # 0.250% = 0.0025
            if R10 >= -0.0025:
              balance = balance + 100 + R10
            # If not profitable, we will close the position and set the currencyCheck flag to false
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False
          else:
            # Short condition -> a profitable trade has a negative return. 
            if R10 <= 0.0025:
              balance = balance + 100 + R10
            # If not profitable, we will close the position and set the currencyCheck flag to false
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False
        
        # At T20, cutoff value to use is 0.150%
        elif window == 2:
          # The below line fetches the last balance of the currency pair of last 60 min period
          result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_output WHERE window = " + str(window-1)+";"))
          for row in result:
            balance = row.balance

          if position == 'LONG':
            if R10 >= -0.0015:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False
          else:
            # Short condition
            if R10 <= 0.0015:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False


        # At T30, value to use is 0.100%
        elif window == 3:
          result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_output WHERE window = " + str(window-1)+";"))
          for row in result:
            balance = row.balance

          if position == 'LONG':
            if R10 >= -0.001:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False
          else:
            if R10 <= 0.001:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False


        # At T40, value to use is 0.050%
        elif window == 4:
          result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_output WHERE window = " + str(window-1)+";"))
          for row in result:
            balance = row.balance

          if position == 'LONG':
            if R10 >= -0.0005:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False
          else:
            if R10 <= 0.0005:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False


        # After T40, value to use is 0.050%
        elif window > 4:
          result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_output WHERE window = " + str(window-1)+";"))
          for row in result:
            balance = row.balance

          if position == 'LONG':
            if R10 >= -0.0005:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False
          else:
            if R10 <= 0.0005:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
#               currencyCheck[curr[0]+curr[1]] = False
          
# Based on the predicted return, decide whether to continue investing
            # At T10, cutoff value to use is 0.250%
        if window == 1:
          balance_pre = 100
          if position == 'LONG':
            # Long condition -> , a profitable trade has a positive return but we have a tolarence of 0.250%
            # 0.250% = 0.0025
            if R10_predict >= -0.0025:
              balance_pre = balance_pre + 100 + R10_predict
            # If not profitable, we will close the position and set the currencyCheck flag to false
            else:
              balance_pre = balance_ + R10_predict
#               currencyCheck[curr[0]+curr[1]] = False
          else:
            # Short condition -> a profitable trade has a negative return. 
            if R10_predict <= 0.0025:
              balance_pre = balance + 100 + R10_predict
            # If not profitable, we will close the position and set the currencyCheck flag to false
            else:
              balance_pre = balance + R10_predict
#               currencyCheck[curr[0]+curr[1]] = False
        
        # At T20, cutoff value to use is 0.150%
        elif window == 2:
          # The below line fetches the last balance of the currency pair of last 60 min period
          result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_output WHERE window = " + str(window-1)+";"))
          for row in result:
            balance_pre = row.balance_pre

          if position == 'LONG':
            if R10_predict >= -0.0015:
              balance_pre = balance_pre + 100 + R10_predict
            else:
              balance_pre = balance_pre + R10_predict
#               currencyCheck[curr[0]+curr[1]] = False
          else:
            # Short condition
            if R10_predict <= 0.0015:
              balance_pre = balance_pre + 100 + R10_predict
            else:
              balance_pre = balance_pre + R10_predict
#               currencyCheck[curr[0]+curr[1]] = False


        # At T30, value to use is 0.100%
        elif window == 3:
          result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_output WHERE window = " + str(window-1)+";"))
          for row in result:
            balance_pre = row.balance_pre

          if position == 'LONG':
            if R10_predict >= -0.001:
              balance_pre = balance_pre + 100 + R10_predict
            else:
              balance_pre = balance_pre + R10_predict
#               currencyCheck[curr[0]+curr[1]] = False
          else:
            if R10_predict <= 0.001:
              balance_pre = balance_pre + 100 + R10_predict
            else:
              balance_pre = balance_pre + R10_predict
#               currencyCheck[curr[0]+curr[1]] = False


        # At T40, value to use is 0.050%
        elif window == 4:
          result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_output WHERE window = " + str(window-1)+";"))
          for row in result:
            balance_pre = row.balance_pre

          if position == 'LONG':
            if R10_predict >= -0.0005:
              balance_pre = balance_pre + 100 + R10_predict
            else:
              balance_pre = balance_pre + R10_predict
#               currencyCheck[curr[0]+curr[1]] = False
          else:
            if R10_predict <= 0.0005:
              balance_pre = balance_pre + 100 + R10_predict
            else:
              balance_pre = balance_pre + R10_predict
#               currencyCheck[curr[0]+curr[1]] = False


        # After T40, value to use is 0.050%
        elif window > 4:
          result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_output WHERE window = " + str(window-1)+";"))
          for row in result:
            balance_pre = row.balance_pre

          if position == 'LONG':
            if R10_predict >= -0.0005:
              balance_pre = balance_pre + 100 + R10_predict
            else:
              balance_pre = balance_pre + R10_predict
#               currencyCheck[curr[0]+curr[1]] = False
          else:
            if R10_predict <= 0.0005:
              balance_pre = balance_pre + 100 + R10_predict
            else:
              balance_pre = balance_pre + R10_predict
#               currencyCheck[curr[0]+curr[1]] = False



        conn.execute(text("INSERT INTO " +curr[0]+curr[1]+"_output (window, balance, returnVal, position, balance_pre, returnVal_predict) VALUES (:window, :balance, :returnVal, :position, :balance_pre, :returnVal_predict);"),[{"window": window, "balance": balance, "returnVal": R10, "position": position, "balance_pre": balance_pre, "returnVal_predict": R10_predict}])

  return currencyCheck

In [42]:
# This main function repeatedly calls the polygon api every 1 seconds for 24 hours 
# and stores the results.
def main1(currency_pairs):
    # The api key given by the professor is fetched from the library imported in the 1st cell 
    key = "beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq"
    
    # Number of list iterations - each one should last about 1 second
    count = 0
    agg_count = 0
    # Counter to keep track of 10 intervals which is after every 60 minutes
    decision_count = 0

    # currencyCheck dictionary has keys as currencyPair and values as True or False
    # By default all are set to True but if we decide not to invest more and stop the position, we set this to False
    currencyCheck = {}
    
    # Create an engine to connect to the database; setting echo to false should stop it from logging in std.out
    engine = create_engine("sqlite:///final1.db", echo=False, future=True)
    
    # Create the needed tables in the database
    initialize_raw_data_tables(engine,currency_pairs)
    initialize_aggregated_tables(engine,currency_pairs)
    # Create the output tables for all currency pairs
    initialize_output_tables(engine,currency_pairs)
    
    # Open a RESTClient for making the api calls
    client = RESTClient(key)
   
    # counter for period 
    period_count = 0 
    # counter for window 
    window_count = 0
    # dictionary to store N values for all currency pairs
    N = {}
    UB = {}
    LB = {}

    # stores last price of currencypairs
    lastPrice = {}

    # Loop that runs until the total duration of the program hits 10 hours. 
    # 36000 seconds = 10 hours
    while count < 36000:
      
      # Make a check to see if 6 minutes has been reached or not
      if agg_count == 360:
        # Aggregate the data and clear the raw data tables
        period_count+=1
        UB, LB = aggregate_raw_data_tables(engine, currency_pairs, period_count, N, currencyCheck)
        reset_raw_data_tables(engine,currency_pairs)
        agg_count = 0
        N = {}
        lastPrice = {}
        
      # Only call the api every 1 second, so wait here for 0.75 seconds, because the 
      # code takes about .15 seconds to run
      time.sleep(0.55)

      # Only called when 10 periods of 6 mins are done i.e after every 60 mins
      if decision_count == 3600:
        window_count +=1
        currencyCheck = fill_output_data_tables(engine, currency_pairs, window_count, currencyCheck)
        decision_count = 0
      
      # Increment the counters
      count += 1
      agg_count +=1
      decision_count +=1

      # Loop through each currency pair
      for currency in currency_pairs:
        # Set the input variables to the API
        from_ = currency[0]
        to = currency[1]

        # initializing currencyCheck for all currency pair with True
        if from_+to not in currencyCheck:
          currencyCheck[from_+to] = True

        # Run the below logic only when currencyCheck of the corresponding currency pair is set to True 
        if currencyCheck[from_+to] == True:
          # Call the API with the required parameters
          try:
              resp = client.forex_currencies_real_time_currency_conversion(from_, to, amount=100, precision=2)
          except:
              continue

          # This gets the Last Trade object defined in the API Resource
          last_trade = resp.last

          # Format the timestamp from the result
          dt = ts_to_datetime(last_trade["timestamp"])

          # Get the current time and format it
          insert_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
          
          # Calculate the price by taking the average of the bid and ask prices
          avg_price = (last_trade['bid'] + last_trade['ask'])/2

          # In the 1st period, UB and LB will be an empty dictionary so this condition will be false for 1st period
          if UB!={} and LB!={}:

            # From 2nd period to 100th period
            # if currency[0]+currency[1] key is in lastPrice, it means we have a last price to send in the countBandsCrossing() function and we update lastPrice with the current price which is the avg_price
            if currency[0]+currency[1] in lastPrice:
              N[currency[0]+currency[1]] += countBandsCrossing(lastPrice[currency[0]+currency[1]], avg_price, UB[currency[0]+currency[1]], LB[currency[0]+currency[1]])    
              lastPrice[currency[0]+currency[1]] = avg_price
            
            # if currency[0]+currency[1] key not in lastPrice and N, we initialize here
            else:
              lastPrice[currency[0]+currency[1]] = avg_price
              N[currency[0]+currency[1]] = 0

          # Write the data to the SQLite database, raw data tables
          with engine.begin() as conn:
              conn.execute(text("INSERT INTO "+from_+to+"_raw(ticktime, fxrate, inserttime) VALUES (:ticktime, :fxrate, :inserttime)"),[{"ticktime": dt, "fxrate": avg_price, "inserttime": insert_time}])

In [43]:
# A dictionary defining the set of currency pairs we will be pulling data for
currency_pairs = [["EUR","USD",[],portfolio("EUR","USD")],
                  ["GBP","USD",[],portfolio("GBP","USD")],
                  ["USD","CHF",[],portfolio("USD","CHF")],
                  ["USD","CAD",[],portfolio("USD","CAD")],
                  ["USD","HKD",[],portfolio("USD","HKD")],
                  ["USD","AUD",[],portfolio("USD","AUD")],
                  ["USD","NZD",[],portfolio("USD","NZD")],
                  ["USD","SGD",[],portfolio("USD","SGD")]]

# Run the main data collection loop
main1(currency_pairs)

OperationalError: (sqlite3.OperationalError) table EURUSD_raw already exists
[SQL: CREATE TABLE EURUSD_raw(ticktime text, fxrate  numeric, inserttime text);]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [44]:
conn = sqlite3.connect('final1.db', isolation_level=None,
                       detect_types=sqlite3.PARSE_COLNAMES)
# make the first return = 0
# Code to convert db tables into csv files


for curr in currency_pairs:
  sql = "SELECT * FROM "+curr[0]+curr[1]+"_agg"
  db_df = pd.read_sql_query(sql, conn)
  path = curr[0]+curr[1]+'_agg_predict_10hour.csv'
  db_df.to_csv(path, index=False)

for curr in currency_pairs:
  sql = "SELECT * FROM "+curr[0]+curr[1]+"_output"
  db_df = pd.read_sql_query(sql, conn)
  path = curr[0]+curr[1]+'_output_predict_10hour.csv'
  db_df.to_csv(path, index=False)